In [ ]:

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, datasets



In [ ]:
# Data
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# num_samples = 10000
num_samples = 50000
train_images = train_images.astype('float32')[:num_samples]
test_images = test_images.astype('float32')
train_images /= 255.0
test_images /= 255.0
train_labels = keras.utils.to_categorical(train_labels, 10)[:num_samples]
test_labels = keras.utils.to_categorical(test_labels, 10)


170498071/170498071 [==============================] - 4s 0us/step


In [ ]:

# Model

input_tensor = layers.Input(shape=(32,32,3))
x = layers.Conv2D(filters=32, kernel_size=(4,4), activation='relu')(input_tensor)
x = layers.MaxPool2D(pool_size=(2,2), padding='same')(x)
x = layers.Conv2D(filters=32, kernel_size=(4,4), activation='relu')(x)
x = layers.MaxPool2D(pool_size=(2,2), padding='same')(x)
x = layers.Conv2D(filters=64, kernel_size=(4,4), activation='relu')(x)
x = layers.Flatten()(x)
output_tensor = layers.Dense(units=10, activation='softmax')(x)

model = keras.Model(inputs=[input_tensor], outputs=[output_tensor])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 29, 29, 32)        1568      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 12, 12, 32)        16416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 64)          32832 

In [ ]:
# Loss, Optimizer, Compile

loss_fun = keras.losses.CategoricalCrossentropy()

model.compile(loss=loss_fun, optimizer='rmsprop', metrics=['accuracy']) # sgd, adam, rmsprop


In [ ]:
'''
a function that takes an epoch index (integer, indexed from 0) and current learning rate (float) as inputs and returns a new learning rate as output (float).
'''
def scheduler(epoch, lr):
    # epochs:100 0.1 ---> 0.0001...
    lr_params = {
        'total_sample_num': 50_000,
        'epochs': 100,
        'batch_size': 128,
        'initial_lr': 0.1,
        'decay_steps': 120 * 2.5,
        'decay_rate': 0.95
    }
    cur_step = int(lr_params['total_sample_num'] / lr_params['batch_size']) * epoch
    initial_lr = lr_params['initial_lr']
    decay_steps = lr_params['decay_steps']
    decay_rate = lr_params['decay_rate']
    
    p = cur_step / decay_steps
    return initial_lr * decay_rate ** p

callback = keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
# Training
    
history = model.fit(train_images,
                    train_labels,
                    validation_data=(test_images, test_labels),
                    epochs=100,
                    batch_size=128,
                    callbacks=[callback],
                    )


Epoch 1/100
391/391 [==============================] - 70s 178ms/step - loss: 1784.6525 - accuracy: 0.1017 - val_loss: 2.3082 - val_accuracy: 0.1000 - lr: 0.1000
Epoch 2/100
391/391 [==============================] - 58s 149ms/step - loss: 2.9504 - accuracy: 0.0984 - val_loss: 2.3206 - val_accuracy: 0.1001 - lr: 0.0935
Epoch 3/100
391/391 [==============================] - 62s 157ms/step - loss: 2.3088 - accuracy: 0.1009 - val_loss: 2.3100 - val_accuracy: 0.1000 - lr: 0.0875
Epoch 4/100
391/391 [==============================] - 60s 152ms/step - loss: 2.3084 - accuracy: 0.0989 - val_loss: 2.3102 - val_accuracy: 0.1000 - lr: 0.0819
Epoch 5/100
391/391 [==============================] - 58s 149ms/step - loss: 2.3075 - accuracy: 0.1019 - val_loss: 2.3056 - val_accuracy: 0.1000 - lr: 0.0766
Epoch 6/100
391/391 [==============================] - 59s 150ms/step - loss: 2.3079 - accuracy: 0.0974 - val_loss: 2.3087 - val_accuracy: 0.1000 - lr: 0.0716
Epoch 7/100
391/391 [======================

In [ ]:
# Examine training result
print(history.history.keys())
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)